In [1]:
from konlpy.tag import Okt
import numpy as np
import os, re, kss
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from hanspell import spell_checker

DATA_PATH = './somelight_2.csv' # somelight_1.csv 중 처음부터 50개 데이터
encoding = 'CP949'
print("File Size: "+str(round(os.path.getsize(DATA_PATH)/1000000, 2))+"MB")

dataset = pd.read_csv(DATA_PATH, header=0, encoding=encoding)
dataset.describe()

File Size: 0.03MB


,결과
count,50.000000
mean,0.860000
std,0.947822
min,0.000000
25%,0.000000
50%,0.000000
75%,2.000000
max,2.000000


In [2]:
okt = Okt() 

stop_word = pd.read_csv('./ko_stopword.txt', header=0)
stop_words = set(stop_word)

In [32]:
def preprocessing(text, okt, re_stopword = True):
    spelled_sent = spell_checker.check(text)
    hanspell_sent = spelled_sent.checked
    retext_1 = re.sub("[a-zA-Zㄱ-ㅎㅏ-ㅣ!?.]", "", hanspell_sent)
    retext_2 = okt.morphs(retext_1, stem=True)
    
    if re_stopword:
        clean_text = [token for token in retext_2 if not token in stop_words]
    
    clean_text = ' '.join(clean_text)
    return clean_text

In [10]:
clean_sent = []
for re_text in dataset['본문']:
    if type(re_text) == str:
        clean_text=[]
        for sent in kss.split_sentences(re_text):
            clean_text += preprocessing(sent, okt)
        clean_sent.append(clean_text)
    else:
        clean_sent.append([])

In [31]:
word_index = []
for sent in clean_sent:
    word_to_index = {word: index for index, word in enumerate(sent)}
    word_index.append(word_to_index)

In [46]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_sent)
encoded = tokenizer.texts_to_sequences(clean_sent)

In [72]:

padded = pad_sequences(encoded, padding='post', maxlen = max(len(item) for item in encoded))
type(padded)

numpy.ndarray

In [76]:
padded.shape

(50, 786)